In [1]:
import yfinance as yf

def get_current_stock_price(ticker_symbol):

  # Get the stock data
  stock = yf.Ticker(ticker_symbol)

  # Get the current price
  current_price = stock.history(period='1d')['Close'].iloc[0]

  return current_price

In [5]:
import ollama

# Want the current price of Microsoft - ticker symbol MSFT

response = ollama.chat(
    model='llama3.1:8b',
    messages=[{'role': 'user','content': 
        'What is the current price of MSFT'}],

  # provide a tool to get the current price of a stock
    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_current_stock_price',
        'description': 'Get the current price for a stock',
        'parameters': {
          'type': 'object',
          'properties': {
            'ticker_symbol': {
              'type': 'string',
              'description': 'The ticker symbol of the stock',
            },
          },
          'required': ['ticker_symbol'],
        },
      },
    },
  ],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_current_stock_price', 'arguments': {'ticker_symbol': 'MSFT'}}}]


In [7]:
# Create a dictionary mapping function names to functions
function_map = {
    'get_current_stock_price': get_current_stock_price,
    # Add more functions here as needed
}

def call_function_safely(response, function_map):
    # Extract the function name and arguments from the response
    tool_call = response['message']['tool_calls'][0]
    print(f"tool_call: {tool_call}")
    
    function_name = tool_call['function']['name']
    print(f"function_name: {function_name}")
    
    arguments = tool_call['function']['arguments']
    print(f"arguments: {arguments}")

    # Look up the function in the function map
    function_to_call = function_map.get(function_name)
    print(f"function_to_call: {function_to_call}")
    
    if function_to_call:
        try:
            # Call the function with the arguments
            result = function_to_call(**arguments)
            print(f"The current price of {arguments['ticker_symbol']} is : {result}")
        except TypeError as e:
            print(f"Argument error: {e}")
    else:
        print(f"{function_name} is not a recognized function")

call_function_safely(response, function_map)

tool_call: {'function': {'name': 'get_current_stock_price', 'arguments': {'ticker_symbol': 'MSFT'}}}
function_name: get_current_stock_price
arguments: {'ticker_symbol': 'MSFT'}
function_to_call: <function get_current_stock_price at 0x1053eb560>
The current price of MSFT is : 417.1400146484375
